In [11]:
from PIL import Image
import cv2
import numpy as np

paa6_sample_image = "D:/code/unet-kedro/data/01_raw/continuous/paa6/1.0026.tiff"
paa12_sample_image = "D:/code/unet-kedro/data/01_raw/image_9.png"

# Read image
image = cv2.imread(paa6_sample_image)

# Create different enhanced versions
# 1. Contrast enhancement
contrast = cv2.convertScaleAbs(image, alpha=1.5, beta=0)

# 2. CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
clahe_img = clahe.apply(gray)

# 3. Histogram equalization
img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
hist_eq = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

# Show all versions
cv2.imshow("Original", image)
cv2.imshow("Contrast Enhanced", contrast)
cv2.imshow("CLAHE", clahe_img)
cv2.imshow("Histogram Equalization", hist_eq)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Save all versions


Histogram equalization appears to perform well. Fine tune with the below code

In [21]:
from PIL import Image
import cv2
import numpy as np
import os
import logging
from pathlib import Path

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create output directories
output_dir = Path("data/02_intermediate/image_enhancements")
output_dir.mkdir(parents=True, exist_ok=True)

def process_and_save_image(image_path):
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    
    # Get base filename without extension
    base_name = Path(image_path).stem
    
    try:
        # 1. Contrast enhancement
        contrast = cv2.convertScaleAbs(image, alpha=1.5, beta=0)
        cv2.imwrite(str(output_dir / f"{base_name}_contrast.png"), contrast)
        
        # 2. CLAHE
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        clahe_img = clahe.apply(gray)
        cv2.imwrite(str(output_dir / f"{base_name}_clahe.png"), clahe_img)
        
        # 3. Histogram equalization
        img_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        hist_eq = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
        cv2.imwrite(str(output_dir / f"{base_name}_hist_eq.png"), hist_eq)
        
        # Save original for reference
        cv2.imwrite(str(output_dir / f"{base_name}_original.png"), image)
        
        # Optional: Display images
        cv2.imshow("Original", image)
        cv2.imshow("Contrast Enhanced", contrast)
        cv2.imshow("CLAHE", clahe_img)
        cv2.imshow("Histogram Equalization", hist_eq)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        logger.info(f"Successfully processed and saved enhancements for {base_name}")
        
    except Exception as e:
        logger.error(f"Error processing {base_name}: {str(e)}")
        raise

# Process both sample images
sample_images = [
    "D:/code/unet-kedro/data/01_raw/continuous/paa6/1.0026.tiff",
    "D:/code/unet-kedro/data/01_raw/image_9.png"
]

for img_path in sample_images:
    try:
        process_and_save_image(img_path)
    except Exception as e:
        logger.error(f"Failed to process {img_path}: {str(e)}")

INFO:__main__:Successfully processed and saved enhancements for 1.0026
INFO:__main__:Successfully processed and saved enhancements for image_9
